In [3]:
import matplotlib.pyplot as plt
import scipy
from scipy import stats
from scipy.stats import norm
import pandas as pd
import numpy as np
import seaborn as sns
from statsmodels.distributions.empirical_distribution import ECDF
from copulae1 import *
from KDEs import *
from toolbox import *
import warnings
import itertools
import os

warnings.filterwarnings("ignore")

data = pd.read_csv('../processed_data/btc_future_crix.csv')
data.head()

ecdf_brr = ECDF(data.return_brr)
ecdf_btc = ECDF(data.return_btc)

u = ecdf_brr(data.return_brr)
v = ecdf_btc(data.return_btc)

brr = np.array(data.return_brr)
btc = np.array(data.return_btc)

kde_brr = KDE(data.return_brr, "Gaussian")
kde_btc = KDE(data.return_btc, "Gaussian")

kde_brr = KDE(data.return_brr, "Gaussian", kde_brr.h_brot*3)
kde_btc = KDE(data.return_brr, "Gaussian", kde_btc.h_brot*3)

In [4]:
# Gaussian, t_Copula, Clayton, Frank, Gumbel, Plackett, Gaussian mix Indep
C1  = Gaussian(dict(rho=0.9),       Law_RS=kde_brr, Law_RF=kde_btc) # fix the maringals!
C2  = t_Copula(dict(rho=0.1, nu=4), Law_RS=kde_brr, Law_RF=kde_btc, nu_lowerbound=2) 
C2c = t_Copula(dict(rho=0.1, nu=4), Law_RS=kde_brr, Law_RF=kde_btc, nu_lowerbound=4) 
C3  = Clayton(dict(theta=0.1),      Law_RS=kde_brr, Law_RF=kde_btc)
C4  = Frank(dict(theta=0.1),        Law_RS=kde_brr, Law_RF=kde_btc)
C5  = Gumbel(dict(theta=3),         Law_RS=kde_brr, Law_RF=kde_btc)
C6  = Plackett(dict(theta=10),      Law_RS=kde_brr, Law_RF=kde_btc)
C7  = Gaussian_Mix_Independent(dict(rho=.9,p=0.7),Law_RS=kde_brr, Law_RF=kde_btc)
Copulae_names = ['Gaussian', 't_Copula', 't_Copula_Capped', 'Clayton', 'Frank', 'Gumbel', 'Plackett', 'Gauss Mix Indep']
Copulae_arr   = [C1,C2,C2c,C3,C4,C5,C6,C7]
Copulae = dict(zip(Copulae_names, Copulae_arr))

In [ ]:
results = []
ls = os.listdir('../processed_data/crix_brr/train/')
ls = [l for l in ls if l.endswith('.csv')]
for file in ls:
    print(file)
    train = pd.read_csv('../processed_data/crix_brr/train/'+file)
    btc = train.loc[:,'log return crix'].iloc[:-1]
    brr = train.loc[:,'log return brr'].iloc[1:]
    u = ECDF(brr)(brr)
    v = ECDF(btc)(btc)
    plt.scatter(u,v)
    paras = []
    likelihood = []
    for C_name in Copulae:
        Copulae[C_name].canonical_calibrate(u,v)
        print(C_name,'is done.\n')
    
    for C_name in Copulae:
        paras.append((C_name,Copulae[C_name].paras))
        
    for C_name in Copulae:
        ln = Copulae[C_name].dependency_likelihood(u,v)
        likelihood.append((C_name,ln))
        
    results.append([paras, likelihood])

6.csv
Optimization terminated successfully.
         Current function value: -0.205614
         Iterations: 34
         Function evaluations: 73
Gaussian is done.

Optimization terminated successfully.
         Current function value: -0.281130
         Iterations: 87
         Function evaluations: 186
t_Copula is done.

Optimization terminated successfully.
         Current function value: -0.274164
         Iterations: 154
         Function evaluations: 295
t_Copula_Capped is done.

Optimization terminated successfully.
         Current function value: -0.193055
         Iterations: 37
         Function evaluations: 80
Clayton is done.

Optimization terminated successfully.
         Current function value: -0.210579
         Iterations: 39
         Function evaluations: 86
Frank is done.

Optimization terminated successfully.
         Current function value: -0.216332
         Iterations: 35
         Function evaluations: 75
Gumbel is done.

Plackett is done.

Optimization terminated

In [ ]:
c_arr = []
date_range_arr = []
for i, file in enumerate(ls):
    train = pd.read_csv('../processed_data/crix_brr/train/'+file)
    a = pd.DataFrame(results[i][0])
    a.columns = ['Copula_name', 'paras']
    a.index = a.iloc[:,0]
    b = pd.DataFrame(results[i][1])
    b.index = b.iloc[:,0]
    c = pd.concat([a.iloc[:,1],b.iloc[:,1]],axis=1)
    c.columns = ['paras', 'likelihood']
    date_range = train.Date.iloc[-1] +' to ' + train.Date.iloc[0]
    date_range_arr.append(date_range)
    print(date_range)
    c_arr.append(c)

In [ ]:
result = pd.concat(dict(zip(date_range_arr, c_arr)), axis=1)

In [ ]:
result.iloc[:,[i for i in range(len(result.columns)) if i%2!=0]].style.highlight_max(color = 'lightgreen', axis = 0)

In [ ]:
result.iloc[:,[i for i in range(len(result.columns)) if i%2==0]]

In [ ]:
result.iloc[[0,1,2,3,4,5,7],[i for i in range(len(result.columns)) if i%2!=0]].style.highlight_max(color = 'lightgreen', axis = 0)

In [ ]:
kde_brr = KDE(data.return_brr, "Gaussian")
kde_btc = KDE(data.return_btc, "Gaussian")

kde_brr = KDE(data.return_brr, "Gaussian", kde_brr.h_brot*3)
kde_btc = KDE(data.return_brr, "Gaussian", kde_btc.h_brot*3)
kde_brr.plot_density()

In [ ]:
kde_brr.plot_density()

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(data.iloc[:,6])

In [ ]:
np.argmax(data.return_brr)

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(data.return_brr)

In [ ]:
kde_brr.h_brot

In [ ]:
s = Gaussian_Mix_Independent(C7.paras, kde_brr, kde_btc).sample(100000)
plt.figure(figsize=(5,5))
plt.scatter(s[:,0],s[:,1])
plt.title("Samples from Gaussian Mix Independent")
plt.xlabel(r"$R^S$")
plt.ylabel(r"$R^F$")

In [ ]:
h=1
rh = s[:,0] - h*s[:,1]
plt.figure(figsize=(6,5))
plt.scatter(s[:,0],s[:,1],c=rh)
plt.title(r"Samples with Color of $R^h$, $h=1$")
plt.xlabel(r"$R^S$")
plt.ylabel(r"$R^F$")
plt.colorbar()

In [ ]:
h=1
rh = s[:,0] - h*s[:,1]
plt.figure(figsize=(6,5))
plt.scatter(s[:,0],s[:,1],c=ECDF(rh)(rh))
plt.title(r"Samples with Color of $F_{R^h}(R^h)$, $h=1$")
plt.xlabel(r"$R^S$")
plt.ylabel(r"$R^F$")
plt.colorbar()

In [ ]:
h=1
rh = s[:,0] - h*s[:,1]
plt.figure(figsize=(6,5))
plt.scatter(s[:,0],s[:,1],c=ERM_weight(10,ECDF(rh)(rh)))
plt.title(r"Samples with Color of ERM weight k=10")
plt.xlabel(r"$R^S$")
plt.ylabel(r"$R^F$")
plt.colorbar()

In [ ]:
h=1
rh = s[:,0] - h*s[:,1]
plt.figure(figsize=(6,5))
plt.scatter(s[:,0],s[:,1],c=-ERM_weight(10,ECDF(rh)(rh))*rh)
plt.title(r"Samples with Color of Contribution to ERM Measure k=10")
plt.xlabel(r"$R^S$")
plt.ylabel(r"$R^F$")
plt.colorbar()